In [148]:
import itertools
from itertools import *
from copy import copy, deepcopy
from heapq import *
from random import *

from sympy import *

init_printing()

Let $\mathcal{A}$ be our alphabet:

In [4]:
def make_alphabet_entry(i):
    alpha = i + ord('a')
    return chr(alpha),i

A = dict(map(make_alphabet_entry, range(26)))
A.update({' ':26})
inverse_A = {v:k for k,v in A.items()}

A

{' ': 26,
 'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

We define a function `encode` that consumes a string and produces a list of integer in our field:

In [5]:
def encode(s, alphabet=A): return list(map(lambda i: alphabet[i], s))

def decode(e, inverse_alphabet=inverse_A): return "".join(map(lambda i: inverse_alphabet[i], e))

The following plaintext is prose taken from [here]:

[here]:http://www.bartleby.com/209/2.html

In [6]:
with open('plain_text.txt', 'r') as f:
    plain_text = f.read().lower()

remove_chars = [',', '.', ';', ':', '\n', '-', '(', ')', "'", '"']
for rc in remove_chars: 
    plain_text = plain_text.replace(rc, '')

plain_text = "".join(filter(lambda c: not c.isdigit(), plain_text))
m = len(plain_text)
plain_text, m

('the attraction of medieval literature comes perhaps more stronglyfrom some other countries than from england in france and provencein germany and iceland there were literary adventurers more daringand achievements more distinguished it was not in england that themost wonderful things were produced there is nothing in old englishthat takes hold of the mind with that masterful and subduing powerwhich still belongs to the lyrical stanzas of the troubadours andminnesingers to welsh romance or to the epic prose of the icelandhistories   the norman conquest degraded the english language fromits literary rank and brought in a new language for the politerliterature it did not destroy in one sense it did not absolutelyinterrupt english literature but it took away the english literarystandard and threw the country back into the condition of italybefore dantean anarchy of dialects when a new literary languagewas established in the time of chaucer the middle ages were nearlyover and so it happen

In [7]:
assert decode(encode(plain_text)) == plain_text

In [8]:
encoded_plain_text = encode(plain_text)

Let $k \in \left(\frac{\mathbb{Z}}{n\mathbb{Z}}\right)^{m}$ be a *key* of length $m\in\mathbb{N}$, for example:

In [150]:
def generate_random_key(alphabet=A):
    length = randint(0,floor(len(alphabet)/2))
    key = [0] * length
    for i in range(length): key[i] = randint(0, len(alphabet)-1)
    return key

# it should be better to randomly generate it to keep it safe
#k = encode("ericsmullyan")
k = generate_random_key(A)
#k, len(k)

Now we are in the position to define the `encrypt` function:

In [151]:
def encrypt(message, key, alphabet=A):
    n = len(alphabet)
    return [(p+v)%n for p,v in zip(message, cycle(key))]

def decrypt(cipher, key, alphabet=A):
    n = len(alphabet)
    return [(c-v)%n for c,v in zip(cipher, cycle(key))]

In [152]:
cipher_text = encrypt(encoded_plain_text, k)
decode(cipher_text)

'lwghpi opvhkwbpvcoetfqtkhiodxvmfq rfxoewauzxdxfjidhgjcjtb hgvkvdmhzcbgpcetbwhxloovcwvhgpbgshjibpmoceogvvahkssxphughkrxocvspwocntpktyuxvxfoibngybwokktahksshjmfugttjtbtxilopjmbisklkhmfgzgptlfxofifyudpfsbirxpbjxagvhhgjcjtblxh fbzik wukxxloyigpulhsxphtcnipfsbawq xh towgigtcfsgzujsxh xpogpcbfxorzcta twovptglxxkopwhxpkvsxphcakxtfvnqgx eplovizuzxwg fhcvgqwxooqbtgtxlwbawq xatgvmfvaiotbfhgjiaiabihddcbfowkkwpzqxd bjtavkvkovwoiobodmtqrqsxglppgphglushjmoiyliupfwigzxpfsoqbclpxfvgzgp lootn wpylatbemodyxhgovptplmxvorzchlxcyovptpp tdpplwyzqcjxg opgqwxopwfbhkovcpyiuzqowtizptlaolwghtcnixkwbtpcnrpztbnfdtfhkonqhuyyfqotib gybwodzcjnehsxphppubks cvvjhdtsuqzoiobohcnqhuyixlttihjyboahblxtgkclofmgiylmsxphcclxgxbumoy xsasbvcigyqkcnbhusvxfhgzfjwqoxbitxhox ahgzpiaotsqwaoy xhgcmhplhvolwghtcnixkwbtxilopjmuapckyfwocvsp efxkbawug cmbvzmpiyrcokvhdgqwxoewbtpqxgbbwuppqpdmdmudybowppatquxpfptkwnglusski ujqgskjmbphxbxkbtxilopjmbtpcnrpztyigplphtqnqgxlaoabbawugqxetbwupjepmrgzoioboexfl ugyvxgbdtglxbxpttmdbbfspplohvxxlojidelkt

In [153]:
assert decode(decrypt(cipher_text, k)) == plain_text

In [154]:
def frequency(elem, lst):
    return lst.count(elem)

def coincidence_index(lst, alphabet=A):
    sums = 0
    freqs = {}
    denom = len(lst)
    #if denom == 0 or denom == 1: return 0,{}
    for k,v in alphabet.items(): freqs[k] = frequency(v, lst)
    ci = sum(v*(v-1) for v in freqs.values())/(denom*(denom-1))
    return ci, freqs

def mutual_coincidence_index(fst, snd, g=0, alphabet=A):
    fst_ci = coincidence_index(fst, alphabet)[1]
    snd_ci = coincidence_index(snd, alphabet)[1]
    return sum(fst_ci[k] * snd_ci[decode([(v+g)%len(alphabet)])] 
               for k,v in alphabet.items())/(len(fst)*len(snd))

In [155]:
coincidence_index(encoded_plain_text)

(0.07593726979645456,
 {' ': 305,
  'a': 115,
  'b': 19,
  'c': 40,
  'd': 67,
  'e': 215,
  'f': 34,
  'g': 39,
  'h': 81,
  'i': 113,
  'j': 0,
  'k': 9,
  'l': 57,
  'm': 29,
  'n': 140,
  'o': 120,
  'p': 21,
  'q': 1,
  'r': 125,
  's': 93,
  't': 162,
  'u': 35,
  'v': 14,
  'w': 29,
  'x': 0,
  'y': 26,
  'z': 1})

In [156]:
coincidence_index(cipher_text)

(0.04518277636329516,
 {' ': 49,
  'a': 58,
  'b': 106,
  'c': 79,
  'd': 37,
  'e': 22,
  'f': 70,
  'g': 115,
  'h': 120,
  'i': 60,
  'j': 38,
  'k': 67,
  'l': 72,
  'm': 40,
  'n': 27,
  'o': 135,
  'p': 132,
  'q': 62,
  'r': 19,
  's': 49,
  't': 105,
  'u': 67,
  'v': 70,
  'w': 77,
  'x': 127,
  'y': 47,
  'z': 40})

In [157]:
def spread(message, block_length):
    return [message[i:i+block_length] for i in range(0, len(message), block_length)]
    
def col(spreaded, c, joiner=""):
    column = [lst[c] if c < len(lst) else None for lst in spreaded]
    ready = list(filter(lambda i: i is not None, column))
    return joiner.join(ready) if joiner is not None else ready

In [158]:
spreaded = spread(cipher_text, 12)

In [159]:
c = col(spreaded,2,None)

In [163]:
def analyze(cipher_text):
    res = {}
    # we discard the case where the key length equals the
    # length of the cipher text, since it is the case of 
    # OneTimePad chiper, which is unbreakable!
    for d in range(2, len(cipher_text)):
        spreaded = spread(cipher_text, d)
        res[d] = []
        for c in range(d):
            column = col(spreaded, c, joiner=None)
            if len(column) < 2: continue
            mci = coincidence_index(column)[0]
            res[d].append(mci)
    return res

def guess_key_length(analysis, threshold=0.06):
    candidates = {}
    for k,v in analysis.items():
        cs = list(filter(lambda i: i > threshold, v))
        if cs and len(cs) > k/2: candidates[k] = cs
    return candidates

In [164]:
analysis = analyze(cipher_text)

In [172]:
probing_key_length = 9
guess_key_length(analysis)[probing_key_length]

In [167]:
def normalize_heap(heap):
    
    def normalize(pair): 
        p,d = pair
        return d, 1-p
    
    return list(map(normalize, heap))

def build_offsets_eqs(cipher_text, key_length, indexed_sym, threshold=.06, alphabet=A):
    
    n = len(alphabet)
    eqs = {c:[] for c in range(key_length)}
    spreaded = spread(cipher_text, key_length)
    for c,a in itertools.product(range(key_length), repeat=2):
    
        if a == c: continue
        
        column = col(spreaded, c, joiner=None)
        another = col(spreaded, a, joiner=None)
        for g in range(1,n):
            mci = mutual_coincidence_index(column, another, g)
            if mci > threshold: 
                eqs[c].append(tuple([a, Eq(indexed_sym[a]-indexed_sym[c],g,evaluate=True), mci]))
    
    return eqs

In [173]:
k_sym=IndexedBase('k')

eqs_dict = build_offsets_eqs(cipher_text, key_length=probing_key_length, indexed_sym=k_sym)

In [175]:
eqs_dict[0]

In [176]:
def clean_mci_info(eqs_dict):
    return {k:list(map(lambda pair: (pair[0], pair[1]), v)) for k,v in eqs_dict.items()}

In [177]:
eqs_dict_pure = clean_mci_info(eqs_dict)
eqs_dict_pure

In [178]:
def candidate_keys(eqs_dict, indexed_sym, apply_mod=False, alphabet=A):
    key_length = len(eqs_dict)
    n = len(alphabet)
    candidates = {}
    for c, eq_pairs in eqs_dict.items():
        candidates[c]=[]
        for d in range(len(alphabet)):
            key = [indexed_sym[i] for i in range(key_length)]
            key[c] = d
            for a, eq in eq_pairs:
                subs_eq = eq.subs(indexed_sym[c],d)
                key[a] = solve(subs_eq, indexed_sym[a])[0]
                if apply_mod: key[a] = key[a] % n
                    
            at_least_one_missing = next((ind for ind in key 
                                         if isinstance(ind, Indexed) or ind >= n), None)
            if not at_least_one_missing: candidates[c].append(key)
    return candidates

In [179]:
possible_keys = candidate_keys(eqs_dict_pure, k_sym)

In [180]:
def attempt_keys(candidate_keys, cipher_text, threshold=.06):
    sols = set()
    flat_keys = []
    for c, keys in candidate_keys.items(): 
        for key in keys: flat_keys.append(key)
    for key in flat_keys:
        decrypted = decrypt(cipher_text, key)
        cipher_check = encrypt(decrypted, key)
        ci = coincidence_index(decrypted)[0]
        ci_check = coincidence_index(cipher_check)[0]
        if ci > threshold:# and ci_check > threshold: 
            sols.add((ci, decode(key)))
    return sols

In [181]:
sols = attempt_keys(possible_keys, cipher_text)

In [182]:
sols

{(0.07593726979645456, ' wjpwxoew'),
 (0.07593726979645456, 'axkqxypfx'),
 (0.07593726979645456, 'bylryzqgy'),
 (0.07593726979645456, 'czmsz rhz'),
 (0.07593726979645456, 'd nt asi '),
 (0.07593726979645456, 'lhvahi qh'),
 (0.07593726979645456, 'miwbijari'),
 (0.07593726979645456, 'njxcjkbsj'),
 (0.07593726979645456, 'okydklctk'),
 (0.07593726979645456, 'plzelmdul'),
 (0.07593726979645456, 'qm fmnevm'),
 (0.07593726979645456, 'rnagnofwn'),
 (0.07593726979645456, 'sobhopgxo'),
 (0.07593726979645456, 'tpcipqhyp'),
 (0.07593726979645456, 'uqdjqrizq'),
 (0.07593726979645456, 'vrekrsj r'),
 (0.07593726979645456, 'wsflstkas'),
 (0.07593726979645456, 'xtgmtulbt'),
 (0.07593726979645456, 'yuhnuvmcu'),
 (0.07593726979645456, 'zviovwndv')}

In [202]:
for sol in sols: 
    key = sol[1]
    print("key:({})\nplaintext:\n{}\n\n".format(key, decode(decrypt(cipher_text, encode(key)))))

key:(okydklctk)
plaintext:
ymjefyywfhyntsetkerjinj fqeqnyjwfyzwjehtrjxeujwmfuxertwjexywtslqckwtrextrjetymjwehtzsywnjxeymfsekwtrejslqfsiensekwfshjefsieuwt jshjnseljwrfscefsienhjqfsieymjwjeajwjeqnyjwfwcefi jsyzwjwxertwjeifwnslfsiefhmnj jrjsyxertwjeinxynslznxmjienyeafxestyensejslqfsieymfyeymjrtxyeatsijwkzqeymnslxeajwjeuwtizhjieymjwjenxestymnslensetqiejslqnxmymfyeyfpjxemtqietkeymjernsieanymeymfyerfxyjwkzqefsiexzgiznsleutajwamnhmexynqqegjqtslxeyteymjeqcwnhfqexyfsdfxetkeymjeywtzgfitzwxefsirnssjxnsljwxeyteajqxmewtrfshjetweyteymjejunheuwtxjetkeymjenhjqfsimnxytwnjxeeeymjestwrfsehtsvzjxyeijlwfijieymjejslqnxmeqfslzfljekwtrnyxeqnyjwfwcewfspefsiegwtzlmyensefesjaeqfslzfljektweymjeutqnyjwqnyjwfyzwjenyeiniestyeijxywtcensetsjexjsxjenyeiniestyefgxtqzyjqcnsyjwwzuyejslqnxmeqnyjwfyzwjegzyenyeyttpefafceymjejslqnxmeqnyjwfwcxyfsifwiefsieymwjaeymjehtzsywcegfhpensyteymjehtsinyntsetkenyfqcgjktwjeifsyjfsefsfwhmcetkeinfqjhyxeamjsefesjaeqnyjwfwceqfslzfljafxejxyfgqnxmjienseymjeynrjetkehmfzhjweymjerniiqjefljxeajwjesj

In [204]:
decode(k)

'tpcipqhyp'

**:)**

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Vigenere cryptoanalysis</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="mailto:massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/scratchpad/blob/master/chapters/crypto/vigenere-cryptoanalysis.ipynb" rel="dct:source">https://github.com/massimo-nocentini/scratchpad/blob/master/chapters/crypto/vigenere-cryptoanalysis.ipynb</a>.